In [1]:
import seaborn as sns
import pandas as pd
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm
from mpl_toolkits import mplot3d
import cv2
import time

In [2]:
BEDFORM = 'rocks'
save_location =  "../FLIR_Camera"
data_filename = BEDFORM + "_fastFlow_ShallowH_test1.pickle"
height_figs_location  = "../FLIR_Camera/"+BEDFORM+"_fastFlow_ShallowH_test1_centeredCam_flume_LookAngle_35Deg_HEIGHT_FIGURES"
slopeMag_figs_location  = "../FLIR_Camera/"+BEDFORM+"_fastFlow_ShallowH_test1_centeredCam_flume_LookAngle_35Deg_SLOPEMAG_FIGURES"
pdfSlope_figs_location = "../FLIR_Camera/"+BEDFORM+"_fastFlow_ShallowH_test1_centeredCam_flume_LookAngle_35Deg_PDFSLOPE_FIGURES"
pdfHeight_figs_location = "../FLIR_Camera/"+BEDFORM+"_fastFlow_ShallowH_test1_centeredCam_flume_LookAngle_35Deg_PDFHEIGHT_FIGURES"

In [3]:
with open(os.path.join(save_location,data_filename), 'rb') as handle:
    all_data_dict_loaded = pickle.load(handle)

In [4]:
all_height_maps = all_data_dict_loaded['all_height_maps']
all_sx_maps = all_data_dict_loaded['all_sx_maps']
all_sy_maps = all_data_dict_loaded['all_sy_maps']
all_raw_frames_0filter = all_data_dict_loaded['all_raw_frames_0filter'] 

## Plot height map

In [ ]:
for i in tqdm(range(all_height_maps.shape[2])):#all_height_maps.shape[2])):   
    s_x = all_sx_maps[:,:,i]
    s_y = all_sy_maps[:,:,i]
    raw_img_0filter = all_raw_frames_0filter[:,:,i]
    Z = np.real(all_height_maps[:,:,i])

    #subplot(r,c) provide the no. of rows and columns
    fig = plt.figure(figsize =(30, 17)) 
    
    ax0 = fig.add_subplot(2, 3, 1)
    ax0.set_title('Raw image: 0 degree filter', fontsize = 14)
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    im0 = ax0.imshow(raw_img_0filter, cmap = plt.cm.gray, aspect='auto')#, vmin = 0, vmax = 255)
    # Create divider for existing axes instance
    divider0 = make_axes_locatable(ax0)
    # Append axes to the right of ax3, with 20% width of ax2
    cax0 = divider0.append_axes("right", size="10%", pad=0.08)
    # Create colorbar in the appended axes
    # Tick locations can be set with the kwarg `ticks`
    # and the format of the ticklabels with kwarg `format`
    cbar0 = plt.colorbar(im0, cax=cax0)

    ax1 = fig.add_subplot(2, 3, 2)
    ax1.set_title('$\sim \\eta (x,y)$', fontsize = 14)
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    im1 = ax1.imshow(Z, aspect='auto') # vmin = -5, vmax = 5, 
    # Create divider for existing axes instance
    divider1 = make_axes_locatable(ax1)
    # Append axes to the right of ax3, with 20% width of ax2
    cax1 = divider1.append_axes("right", size="10%", pad=0.08)
    # Create colorbar in the appended axes
    # Tick locations can be set with the kwarg `ticks`
    # and the format of the ticklabels with kwarg `format`
    cbar1 = plt.colorbar(im1, cax=cax1)
    # Remove yticks from ax1
    ax1.yaxis.set_visible(False)

    ax2 = fig.add_subplot(2, 3, 3, projection = '3d')
    # get 3D plot data
    rows, cols = Z.shape
    X,Y = np.meshgrid(np.arange(0,cols), np.arange(0,rows))
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    # syntax for plotting 
    p = ax2.plot_surface(Y, X, Z, cmap ='viridis', rstride = 2, cstride = 2)#, edgecolor ='green') 
    # ax.set_title('Height map')
    ax2.set_zlabel('$\\eta$', fontsize = 14)
    ax2.view_init(azim = -61, elev = 52)
    
    ax3 = fig.add_subplot(2, 3, 5)
    ax3.set_title('$s_x:~ tan(\\theta - \\theta_{ref})$', fontsize = 14)
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    im3 = ax3.imshow(s_x, aspect='auto') # vmin = -5, vmax = 5, 
    # Create divider for existing axes instance
    divider3 = make_axes_locatable(ax3)
    # Append axes to the right of ax3, with 20% width of ax2
    cax3 = divider3.append_axes("right", size="10%", pad=0.08)
    # Create colorbar in the appended axes
    # Tick locations can be set with the kwarg `ticks`
    # and the format of the ticklabels with kwarg `format`
    cbar3 = plt.colorbar(im3, cax=cax3)
    # Remove yticks from ax1
    ax3.yaxis.set_visible(True)

    ax4 = fig.add_subplot(2, 3, 6)
    ax4.set_title('$s_y: ~tan(\\phi - \\phi_{ref})$', fontsize = 14)
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    im4 = ax4.imshow(s_y, cmap = plt.cm.cividis,  aspect='auto') # vmin = -5, vmax = 5,
    # Create divider for existing axes instance
    divider4 = make_axes_locatable(ax4)
    # Append axes to the right of ax3, with 20% width of ax2
    cax4 = divider4.append_axes("right", size="10%", pad=0.08)
    # Create colorbar in the appended axes
    # Tick locations can be set with the kwarg `ticks`
    # and the format of the ticklabels with kwarg `format`
    cbar4 = plt.colorbar(im4, cax=cax4)
    # Remove yticks from ax2
    ax4.yaxis.set_visible(False)

    plt.savefig(os.path.join(height_figs_location, 'fig_' + str(i).zfill(6) + '.png'), dpi = 80)

## Plot pdf of the height field

In [5]:
sns.set_style("white")
# Plot
kwargs = dict(hist_kws={'alpha':.6}, kde_kws={'linewidth':2})
xmin = np.min(np.real(all_height_maps))
xmax = np.max(np.real(all_height_maps))

for i in tqdm(range(all_height_maps.shape[2])):#all_height_maps.shape[2]):   
    height_map = all_height_maps[:,:,i]
    Z = np.real(height_map)
    df = pd.DataFrame(Z.ravel())

    #subplot(r,c) provide the no. of rows and columns
    fig = plt.figure(figsize =(40, 15)) 
    
    ax0 = fig.add_subplot(1, 2, 1)
    ax0.set_title('Normalized histogram + KDF ($\eta$)', fontsize = 34)
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    sns.distplot(df, ax=ax0, color="dodgerblue", **kwargs)
    ax0.set_xlabel("$\eta(x,y)$")
    ax0.set_xlim([xmin, xmax])

    ax1 = fig.add_subplot(1, 2, 2)
    ax1.set_title('$\sim \\eta (x,y)$', fontsize = 34)
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    im1 = ax1.imshow(Z, cmap ='viridis', aspect='auto') # vmin = -5, vmax = 5, 
    # Create divider for existing axes instance
    divider1 = make_axes_locatable(ax1)
    # Append axes to the right of ax3, with 20% width of ax2
    cax1 = divider1.append_axes("right", size="10%", pad=0.08)
    # Create colorbar in the appended axes
    # Tick locations can be set with the kwarg `ticks`
    # and the format of the ticklabels with kwarg `format`
    cbar1 = plt.colorbar(im1, cax=cax1)
    # Remove yticks from ax1

    plt.savefig(os.path.join(pdfHeight_figs_location, 'fig_' + str(i).zfill(6) + '.png'), dpi = 40)
    plt.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [01:39<00:00,  1.32s/it]


## Plot pdfs of the slopes

In [6]:
sns.set_style("white")
# Plot
kwargs = dict(hist_kws={'alpha':.6}, kde_kws={'linewidth':2})
sxmin = -1#np.min(all_sx_maps)
sxmax = 1#np.max(all_sx_maps)

symin = -1#np.min(all_sy_maps)
symax = 1#np.max(all_sy_maps)

for i in tqdm(range(all_height_maps.shape[2])):#all_height_maps.shape[2]):   
    height_map = all_height_maps[:,:,i]
    s_x = all_sx_maps[:,:,i]
    s_y = all_sy_maps[:,:,i]
    
    #subplot(r,c) provide the no. of rows and columns
    fig = plt.figure(figsize =(20, 17)) 
    
    df = pd.DataFrame(s_x.ravel())
    ax0 = fig.add_subplot(2, 2, 1)
    ax0.set_title('Normalized histogram + KDF ($s_x$)', fontsize = 24)
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    sns.distplot(df, ax=ax0, color="dodgerblue", **kwargs)
    ax0.set_xlabel("$s_x(x,y)$")
    ax0.set_xlim([sxmin, sxmax])
    
    df = pd.DataFrame(s_y.ravel())
    ax1 = fig.add_subplot(2, 2, 2)
    ax1.set_title('Normalized histogram + KDF ($s_y$)', fontsize = 24)
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    sns.distplot(df, ax=ax1, color="pink", **kwargs)
    ax1.set_xlabel("$s_y(x,y)$")
    ax1.set_xlim([symin, symax])
    
    ax2 = fig.add_subplot(2, 2, 3)
    ax2.set_title('$s_x:~ tan(\\theta - \\theta_{ref})$', fontsize = 24)
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    im2 = ax2.imshow(s_x, cmap = plt.cm.viridis, aspect='auto')#, vmin = 0, vmax = 255)
    # Create divider for existing axes instance
    divider2 = make_axes_locatable(ax2)
    # Append axes to the right of ax3, with 20% width of ax2
    cax2 = divider2.append_axes("right", size="10%", pad=0.08)
    # Create colorbar in the appended axes
    cbar2 = plt.colorbar(im2, cax=cax2)

    ax3 = fig.add_subplot(2, 2, 4)
    ax3.set_title('$s_y:~ tan(\\phi - \\phi_{ref})$', fontsize = 24)
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    im3 = ax3.imshow(s_y, cmap = plt.cm.cividis, aspect='auto') # vmin = -5, vmax = 5, 
    # Create divider for existing axes instance
    divider3 = make_axes_locatable(ax3)
    # Append axes to the right of ax3, with 20% width of ax2
    cax3 = divider3.append_axes("right", size="10%", pad=0.08)
    # Create colorbar in the appended axes
    cbar3 = plt.colorbar(im3, cax=cax3)    

    plt.savefig(os.path.join(pdfSlope_figs_location, 'fig_' + str(i).zfill(6) + '.png'), dpi = 80)
    plt.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [03:19<00:00,  2.62s/it]


## Try HOG aproach : slope magnitude and angle
since we already have s_x  which represents gradient in the flow direction and s_y that represents the gradients in the transverse directiion

In [7]:
for i in tqdm(range(all_height_maps.shape[2])):#all_height_maps.shape[2]):   
    height_map = all_height_maps[:,:,i]
    s_x = all_sx_maps[:,:,i]
    s_y = all_sy_maps[:,:,i]
    # calculate gradient magnitude and direction ( in degrees )
    mag, angle = cv2.cartToPolar(s_y, s_x, angleInDegrees=True)

    #subplot(r,c) provide the no. of rows and columns
    fig = plt.figure(figsize =(25, 17)) 
    
    ax0 = fig.add_subplot(2, 2, 1)
    ax0.set_title('$s_x:~ tan(\\theta - \\theta_{ref})$', fontsize = 24)
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    im0 = ax0.imshow(s_x, cmap = plt.cm.viridis, aspect='auto')#, vmin = 0, vmax = 255)
    # Create divider for existing axes instance
    divider0 = make_axes_locatable(ax0)
    # Append axes to the right of ax3, with 20% width of ax2
    cax0 = divider0.append_axes("right", size="10%", pad=0.08)
    # Create colorbar in the appended axes
    cbar0 = plt.colorbar(im0, cax=cax0)

    ax1 = fig.add_subplot(2, 2, 2)
    ax1.set_title('$s_y:~ tan(\\phi - \\phi_{ref})$', fontsize = 24)
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    im1 = ax1.imshow(s_y, cmap = plt.cm.cividis, aspect='auto') # vmin = -5, vmax = 5, 
    # Create divider for existing axes instance
    divider1 = make_axes_locatable(ax1)
    # Append axes to the right of ax3, with 20% width of ax2
    cax1 = divider1.append_axes("right", size="10%", pad=0.08)
    # Create colorbar in the appended axes
    cbar1 = plt.colorbar(im1, cax=cax1)
    # Remove yticks from ax1
    ax1.yaxis.set_visible(False)
    
    ax2 = fig.add_subplot(2, 2, 3)
    ax2.set_title('$\\sqrt{s_x^2 + s_y^2}$', fontsize = 24)
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    im2 = ax2.imshow(mag, cmap = plt.cm.viridis, aspect='auto', vmin = 0, vmax = np.max(mag)/2) 
    # Create divider for existing axes instance
    divider2 = make_axes_locatable(ax2)
    # Append axes to the right of ax3, with 20% width of ax2
    cax2 = divider2.append_axes("right", size="10%", pad=0.08)
    cbar2 = plt.colorbar(im2, cax=cax2)
    
    ax3 = fig.add_subplot(2, 2, 4)
    ax3.set_title('$arctan(\\frac{s_y}{s_x})$', fontsize = 24)
    # Display image, `aspect='auto'` makes it fill the whole `axes` (ax2)
    im3 = ax3.imshow(angle, cmap = plt.cm.viridis, aspect='auto', vmin = 0, vmax = 360) 
    # Create divider for existing axes instance
    divider3 = make_axes_locatable(ax3)
    # Append axes to the right of ax3, with 20% width of ax2
    cax3 = divider3.append_axes("right", size="10%", pad=0.08)
    # Create colorbar in the appended axes
    cbar3 = plt.colorbar(im3, cax=cax3)
    # Remove yticks from ax1
    ax3.yaxis.set_visible(False)

    plt.savefig(os.path.join(slopeMag_figs_location, 'fig_' + str(i).zfill(6) + '.png'), dpi = 80)
    plt.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [02:31<00:00,  2.01s/it]
